## **Project Goal**

Working on a job listings data scraped from multiple job boards (like LinkedIn, Indeed, Glassdoor, etc.), with the goal to:

- Clean inconsistencies across job sources
- Standardize fields like salary, skills, job titles
- Extract and structure unstructured fields like requirements
- Analyze trends like in-demand skills, top job titles, salary ranges by city

In [0]:
from pyspark.sql.functions import when, col, split, to_date, coalesce, year, month, dayofmonth, trim, lower
from pyspark.sql.functions import explode, array_contains, lit, transform

## 1. Schema Design

I'll work on 4 messy but realistic datasets, ready to be used in Spark.

🟢 Jobs:

- `job_id`: Unique ID for every job listing
- `job_title`: (e.g., “Data Scientist”)
- `post_date`: Date when listing was posted
- `job_type`: (e.g., “Full-Time”, “FT”, “Contract”)
- `city_location`: (e.g., “Toronto, ON”, “Montreal,Quebec”, “Remote”)

🟢 Companies
- `company_id`: Unique ID for every company
- `company_name`: (e.g. “Google Inc”)
- `industry`: (e.g., “Tech”, “Information Technology”)
- `headquarters`:	(e.g., "Mountain View, CA")
- `founded	Year`: The year when company was founded

🟢 Requirements
- `requirements_id`: Unique ID for every reqiremenet or skill
- `experience_years`:	(e.g., “2-4 years”, “3+”, “at least 1”)
- `education_level`:	(e.g., “Bachelor’s”, “B.Sc.”, “BS”, “None”)
- `skills_required`:	(e.g., “Python, SQL, ML, communication”)

🟢 Salaries
- `salary_id`:  Unique ID for every salary range
- `salary_range`:  (e.g. “$60k-$80k/year”, “80,000 per annum”, “Negotiable”)
- `currency`: (e.g., “CAD”, “USD”, “CAD$”)
- `compensation_type`: (e.g., “Annual”, “Hourly”, “Contract”)

## 2. Load Data

Here, it's time to load 4 datasets we have stored in database.

In [0]:
# List all tables in 'default' database
tables = spark.catalog.listTables("default")

# Loop through and read each table into a DataFrame
dfs = {}

for table in tables:
    table_name = table.name
    print(f"Table: {table_name}")
    df = spark.read.table(f"default.{table_name}")
    dfs[table_name] = df


Table: companies
Table: jobs
Table: requirements
Table: salaries


Now, we can store all tables in our database into separate dataframe.

In [0]:
jobs_df = spark.read.table("default.jobs")
companies_df = spark.read.table("default.companies")
requirements_df = spark.read.table("default.requirements")
salaries_df = spark.read.table("default.salaries")

## 3. Data Cleaning Jobs

### 3.1. Handle Null Values

Let's first see top 3 rows of data:

In [0]:
jobs_df.limit(3).display()

job_id,job_title,company_id,post_date,job_type,city_location,requirements_id,salary_id
j00000,Data Engineer,comp164,02-26-2025,Part-Time,Remote,req00000,sal00000
j00001,Data Analyst,comp029,06-24-2024,F/T,"Edmonton, AB",req00001,sal00001
j00002,Data Analyst,comp007,03-24-2025,F/T,"Ottawa, ON",req00002,sal00002


First of all, we can see each column has how many null values:

In [0]:
for COL_NAME in jobs_df.columns:
    null_count = jobs_df.filter(col(COL_NAME).isNull()).count()
    print(f" '{COL_NAME}' has: {null_count} null value")

 'job_id' has: 0 null value
 'job_title' has: 0 null value
 'company_id' has: 0 null value
 'post_date' has: 0 null value
 'job_type' has: 0 null value
 'city_location' has: 0 null value
 'requirements_id' has: 0 null value
 'salary_id' has: 0 null value


### 3.2. Handle Categorical Data

In this table, we have some categorical data as the following:
- `job_title`
- `job_type`
- `city_location`

for these columns, I'll check the unique values to make sure about consistency and reliability.

#### 3.2.1 `job_title`

We should see unique values to make sure about consistency:

In [0]:
print(
    jobs_df.select("job_title") \
                    .distinct() \
                    .sort("job_title") \
                    .rdd.flatMap(lambda x: x) \ #Converts Row objects to plain values.
                    .collect() #Convert to list
)

  File <command-1164924135469545>:5
    .rdd.flatMap(lambda x: x) \ #Converts Row objects to plain values.
                                                                      
^
SyntaxError: unexpected character after line continuation character


There is no issues with this column and we don't have any duplicate category.

#### 3.2.2 `job_type`

Now, we do the same for this column:

In [0]:
print(
    jobs_df.select("job_type") \
                    .distinct() \
                    .sort("job_type") \
                    .rdd.flatMap(lambda x: x) \
                    .collect() 
)

['Contract', 'F/T', 'FT', 'Full Time', 'Internship', 'Part-Time']


It shows we need to marge some categories to have final categories like the below:
- Contract
- Full Time
- Part Time
- Internship

In [0]:
jobs_df = jobs_df.withColumn(
    "job_type",
    when(col("job_type").isin("F/T", "FT"), "Full Time")
    .when(col("job_type").isin("Part-Time"), "Part Time")
    .otherwise(col("job_type"))
)

#### 3.2.3 `city_location`

We should see unique values to make sure about consistency:

In [0]:
print(
    jobs_df.select("city_location") \
                    .distinct() \
                    .sort("city_location") \
                    .rdd.flatMap(lambda x: x) \
                    .collect()
)

['Calgary, AB', 'Edmonton, AB', 'Montreal, QC', 'Ottawa, ON', 'Remote', 'Toronto, ON', 'Vancouver, BC']


Now, we should separate city and province and instead of one column, we can define two columns. city and province.

In [0]:
jobs_df = jobs_df.withColumn("city", split(col("city_location"), ", ")[0]) \
                 .withColumn("province", split(col("city_location"), ", ")[1])

jobs_df = jobs_df.drop("city_location")

Now, we can fill missing values of province for remote location

In [0]:
jobs_df = jobs_df.fillna({"province": "Remote"})

### 3.3. Handle Date Format Data

In this table, we have one date format data as the following:
- `post_date`

For this column, we should make sure data is in date format and then we can segregate it to Year, Month and Day.

In [0]:
jobs_df = jobs_df.withColumn(
    "date_posting",
    coalesce(
        to_date(col("post_date"), "dd-MM-yyyy"),
        to_date(col("post_date"), "MM-dd-yyyy"),
        to_date(col("post_date"), "yyyy/MM/dd"),
        to_date(col("post_date"), "MMM dd yyyy")
    )
)

jobs_df = jobs_df.drop("post_date")

Now, we can split it to Year, month and day.

In [0]:
jobs_df = jobs_df.withColumn("year_post", year(col("date_posting"))) \
                 .withColumn("month_post", month(col("date_posting"))) \
                 .withColumn("day_post", dayofmonth(col("date_posting")))

jobs_df = jobs_df.drop("date_posting")                 

## 4. Data Cleaning Companies

### 4.1. Handle Null Values

Let's first see top 3 rows of data:

In [0]:
Companies_df.limit(3).display()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1527829509962863>:1
----> 1 Companies_df.limit(3).display()

NameError: name 'Companies_df' is not defined

First of all, we can see each column has how many null values:

In [0]:
for COL_NAME in Companies_df.columns:
    null_count = Companies_df.filter(col(COL_NAME).isNull()).count()
    print(f" '{COL_NAME}' has: {null_count} null value")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1362181281344466>:1
----> 1 for COL_NAME in Companies_df.columns:
      2     null_count = Companies_df.filter(col(COL_NAME).isNull()).count()
      3     print(f" '{COL_NAME}' has: {null_count} null value")

NameError: name 'Companies_df' is not defined

We should take a look at `founded` column to deal with null values.

In [0]:
print("Number of total rows:", Companies_df.select('founded').count())
print("Number of null rows in 'founded':", Companies_df.filter(Companies_df['founded'].isNull()).count())

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1362181281344465>:1
----> 1 print("Number of total rows:", Companies_df.select('founded').count())
      2 print("Number of null rows in 'founded':", Companies_df.filter(Companies_df['founded'].isNull()).count())

NameError: name 'Companies_df' is not defined

In this case, we can remove the whole column because it does not have any value for analysis.

In [0]:
Companies_df = Companies_df.drop('founded')

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1362181281344487>:1
----> 1 Companies_df = Companies_df.drop('founded')

NameError: name 'Companies_df' is not defined

### 4.2. Handle Categorical Data

In this table, we have some categorical data as the following:
- `company_name`
- `industry`
- `headquarters`

for these columns, I'll check the unique values to make sure about consistency and reliability.

#### 4.2.1 `company_name`

First, let's take a look at unique categories.

In [0]:
print(
    Companies_df.select("company_name") \
                    .distinct() \
                    .sort("company_name") \
                    .rdd.flatMap(lambda x: x) \
                    .collect() 
)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1164924135469586>:2
      1 print(
----> 2     Companies_df.select("company_name") \
      3                     .distinct() \
      4                     .sort("company_name") \
      5                     .rdd.flatMap(lambda x: x) \
      6                     .collect() 
      7 )

NameError: name 'Companies_df' is not defined

It does not have any duplicate values.

#### 4.2.2 `industry`

Now, let's see this column unique categories:

In [0]:
print(
    Companies_df.select("industry") \
                    .distinct() \
                    .sort("industry") \
                    .rdd.flatMap(lambda x: x) \
                    .collect() 
)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1164924135469592>:2
      1 print(
----> 2     Companies_df.select("industry") \
      3                     .distinct() \
      4                     .sort("industry") \
      5                     .rdd.flatMap(lambda x: x) \
      6                     .collect() 
      7 )

NameError: name 'Companies_df' is not defined

There is  no issue, for this column.

#### 4.2.3 `headquarters`

And now this categorical column:

In [0]:
print(
    Companies_df.select("headquarters") \
                    .distinct() \
                    .sort("headquarters") \
                    .rdd.flatMap(lambda x: x) \
                    .collect() 
)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1164924135469597>:2
      1 print(
----> 2     Companies_df.select("headquarters") \
      3                     .distinct() \
      4                     .sort("headquarters") \
      5                     .rdd.flatMap(lambda x: x) \
      6                     .collect() 
      7 )

NameError: name 'Companies_df' is not defined

And there is no issue for this column, also.

## 5. Data Cleaning Requiremenets

### 5.1. Handle Null Values

Let's first see top 3 rows of data:

In [0]:
requirements_df.limit(3).display()

requirements_id,experience_years,education_level,skills_required
req00000,None,Bachelor's,"Hadoop, Leadership, Python"
req00001,1 year,None,"Communication, PowerBI, Scala"
req00002,1 year,MSc,"Spark, PowerBI"


First of all, we can see each column has how many null values:

In [0]:
for COL_NAME in requirements_df.columns:
    null_count = requirements_df.filter(col(COL_NAME).isNull()).count()
    print(f" '{COL_NAME}' has: {null_count} null value")

 'requirements_id' has: 0 null value
 'experience_years' has: 0 null value
 'education_level' has: 0 null value
 'skills_required' has: 0 null value


### 5.2. Handle Categorical Data

In this table, we have some categorical data as the following:
- `education_level`
- `skill_required`

for these columns, I'll check the unique values to make sure about consistency and reliability.

#### 5.2.1 `education_level`

Let's see difference categories of this column.

In [0]:
print(
    requirements_df.select("education_level") \
                    .distinct() \
                    .sort("education_level") \
                    .rdd.flatMap(lambda x: x) \
                    .collect() 
)

['B.Sc.', 'BS', "Bachelor's", 'MSc', 'None', 'PhD']


We should only have the below categories. So we need to convert some of them:
- Bachelor
- Master
- None
- PhD

In [0]:
requirements_df = requirements_df.withColumn(
    "education_level",
    when(col("education_level").isin("B.Sc.", "BS", "Bachelor's"), "Bachelor")
    .when(col("education_level").isin("MSc"), "Master")
    .otherwise(col("education_level"))
)

#### 5.2.2 `skill_required`

First thing, is that we should split values to some columns. So, if the new column has that skill we put 1 and otherwise 0. So data would be sparse. First of all, we should normalize data:

In [0]:
def one_hot_encode_skills(df, skills_column):
    # Step 1: Split the 'skills_required' column and remove any extra spaces from each individual skill
    df_copy = df.withColumn(
        "skills_array",
        split(trim(lower(col(skills_column))), ",")  # Split by comma and remove extra spaces
    )

    # Trim spaces for each skill individually in the array using transform
    df_copy = df_copy.withColumn(
        "skills_array", 
        transform(col("skills_array"), lambda x: trim(x))  # Apply trim to each element in the array
    )

    # Step 2: Get the unique skills by exploding the 'skills_array' and getting distinct values
    unique_skills = df_copy.select(explode(col("skills_array")).alias("skill")) \
                           .distinct() \
                           .rdd.flatMap(lambda x: x) \
                           .collect()

    # Step 3: One-hot encode for each skill
    for skill in unique_skills:
        df_copy = df_copy.withColumn(
            skill,
            when(array_contains(col("skills_array"), skill), lit(1)).otherwise(lit(0))
        )

    df_copy = df_copy.drop("skills_array")
    df_copy = df_copy.drop(skills_column)
    return df_copy

In [0]:
requirements_df = one_hot_encode_skills(requirements_df, "skills_required")

### 5.3. Handle Numerical Data

Also, we have one numerical data:
- `experience_years`

For this column, we should remove all symbols and just keep the numbers.

First, we should conver some lables:

In [0]:
requirements_df = requirements_df.withColumn(
    "experience_years",
    when(col("experience_years").isin("1 year"), 1)
    .when(col("experience_years").isin("2+" , "at least 2"), 2)
    .when(col("experience_years").isin("3-5 yrs"), 4)
    .when(col("experience_years").isin("None"), 0)
    .otherwise(col("experience_years"))
)

Now, we should change the column type

In [0]:
requirements_df = requirements_df.withColumn(
    'experience_years',  
    col('experience_years').cast('int') 
)

## 6. Data Cleaning Salaries

### 6.1. Handle Null Values

Let's first see top 3 rows of data:

In [0]:
Salaries_df.limit(3).display()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1527829509962857>:1
----> 1 Salaries_df.limit(3).display()

NameError: name 'Salaries_df' is not defined

First of all, we can see each column has how many null values:

In [0]:
for COL_NAME in Salaries_df.columns:
    null_count = Salaries_df.filter(col(COL_NAME).isNull()).count()
    print(f" '{COL_NAME}' has: {null_count} null value")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1362181281344482>:1
----> 1 for COL_NAME in Salaries_df.columns:
      2     null_count = Salaries_df.filter(col(COL_NAME).isNull()).count()
      3     print(f" '{COL_NAME}' has: {null_count} null value")

NameError: name 'Salaries_df' is not defined

### 6.2. Handle Categorical Data

In this table, we have two categorical data as the following:
- `currency`
- `compensation_type`

for these columns, I'll check the unique values to make sure about consistency and reliability.

#### 6.2.1 `currency`

First, let's see the unique categories:

In [0]:
print(
    salaries_df.select("currency") \
                    .distinct() \
                    .sort("currency") \
                    .rdd.flatMap(lambda x: x) \
                    .collect() 
)

['$CAD', 'CAD', 'CAD$', 'USD']


Now, we should convert some categories:

In [0]:
salaries_df = salaries_df.withColumn(
    "currency",
    when(col("currency").isin("$CAD", "CAD$"), "CAD")
    .otherwise(col("currency"))
)

#### 6.2.2 `compensation_type`

We can take a look at unique categories:

In [0]:
print(
    salaries_df.select("compensation_type") \
                    .distinct() \
                    .sort("compensation_type") \
                    .rdd.flatMap(lambda x: x) \
                    .collect() 
)

['Annual', 'Hourly']


There is no issues regarding this column.

### 6.3. Handle Numerical Data

We have one numerical data:
- `salary_range`

For this column, we can split it into two columns has minimum and maximum salary which are numerical data.

In [0]:
print(
    salaries_df.select("salary_range") \
                    .distinct() \
                    .sort("salary_range") \
                    .rdd.flatMap(lambda x: x) \
                    .collect() 
)


['$55K to $75K', '$60k - $80k', '$90,000 - $120,000', '35/hr', '50000 - 70000', '80000 per annum', 'Negotiable']


And, we should convert some lables:

In [0]:
salaries_df = salaries_df.withColumn(
    "salary_range",
    when(col("salary_range").isin("$55K to $75K" , "50000 - 70000"), "$50K - $75K")
    .when(col("salary_range").isin("$90,000 - $120,000"), "$90K - $120K")
    .when(col("salary_range").isin("80000 per annum"), "$80K")
    .when(col("salary_range").isin("35/hr"), "Negotiable")
    .otherwise(col("salary_range"))
)

## 7. Load Cleaned Tables to New Database

Now, we can see final version of cleaned data in different tables:

In [0]:
jobs_df.limit(3).display()

job_id,job_title,company_id,job_type,requirements_id,salary_id,city,province,year_post,month_post,day_post
j00000,Data Engineer,comp164,Part Time,req00000,sal00000,Remote,Remote,2025,2,26
j00001,Data Analyst,comp029,Full Time,req00001,sal00001,Edmonton,AB,2024,6,24
j00002,Data Analyst,comp007,Full Time,req00002,sal00002,Ottawa,ON,2025,3,24


In [0]:
companies_df.limit(3).display()

company_id,company_name,industry,headquarters,founded
comp069,"Wilson, Frost and Robinson",Retail,"West Patricia, NH",null
comp034,Wilson-Mitchell,Education,"New Joshuachester, FL",2000.0
comp112,Chapman-Allen,Healthcare,"Jacobsville, AL",null


In [0]:
requirements_df.limit(3).display()

requirements_id,experience_years,education_level,leadership,excel,tableau,sql,scala,spark,communication,r,powerbi,hadoop,python
req00000,0,Bachelor,1,0,0,0,0,0,0,0,0,1,1
req00001,1,None,0,0,0,0,1,0,1,0,1,0,0
req00002,1,Master,0,0,0,0,0,1,0,0,1,0,0


In [0]:
salaries_df.limit(3).display()

salary_id,salary_range,currency,compensation_type
sal00000,$60k - $80k,CAD,Hourly
sal00001,$60k - $80k,CAD,Hourly
sal00002,$60k - $80k,CAD,Annual


Now, we can push our clean data to a new database. First, I create a new database:

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS clean_database")

Out[82]: DataFrame[]

Now, I push above tables into the new database.

In [0]:
jobs_df.write.mode("overwrite").saveAsTable("clean_database.db_jobs")
companies_df.write.mode("overwrite").saveAsTable("clean_database.db_companies")
requirements_df.write.mode("overwrite").saveAsTable("clean_database.db_requirements")
salaries_df.write.mode("overwrite").saveAsTable("clean_database.db_salaries")

## 8. Exploratory Data Analysis with (Spark SQL)

In this section, I answer some questions to create business values by using PySpark and Spark SQL.

#### `Most Frequent Salary Range for each Job Yype`

In [0]:
%sql
USE clean_database;

SELECT J.job_type, S.salary_range, COUNT(S.salary_range) AS Number_of_Job
FROM db_jobs J
JOIN db_salaries S ON J.salary_id = S.salary_id
GROUP BY J.job_type, S.salary_range
ORDER BY COUNT(S.salary_range) DESC

job_type,salary_range,Number_of_Job
Full Time,$50K - $75K,2874
Full Time,Negotiable,2863
Full Time,$90K - $120K,1469
Full Time,$80K,1443
Full Time,$60k - $80k,1367
Contract,Negotiable,983
Contract,$50K - $75K,965
Part Time,$50K - $75K,958
Part Time,Negotiable,932
Internship,Negotiable,920


#### `Most Number of Job Listing for each Company`

In [0]:
%sql
USE clean_database;

SELECT C.company_name, COUNT(J.job_id) AS Number_of_Job
FROM db_companies C
JOIN db_jobs J ON C.company_id = J.company_id
GROUP BY C.company_name
ORDER BY COUNT(J.job_id) DESC
LIMIT 10

company_name,Number_of_Job
Taylor PLC,216
Simpson LLC,203
Miller Ltd,127
Robbins Group,126
Martin and Sons,126
Williams and Sons,121
"Gonzalez, Ortega and Fisher",121
"Brown, Keith and Arroyo",120
Mathews-Haney,120
Brown-Sawyer,119


#### `Percentage of Jobs require knowledge Python, SQL and Spark`

In [0]:
%sql
USE clean_database;

CREATE OR REPLACE TEMP VIEW PythonSQLSpark AS
SELECT requirements_id, python, sql, spark,
  CASE
    WHEN (python + sql + spark) = 3 THEN 1
    ELSE 0
  END AS skill_match
FROM db_requirements;

SELECT (sum(PSS.skill_match)/count(J.job_id) * 100) AS Percentage_Python_SQL_Spark
FROM db_jobs J
left JOIN PythonSQLSpark PSS ON PSS.requirements_id = J.requirements_id

Percentage_Python_SQL_Spark
2.32
